In [ ]:
import numpy as np
import os
import gc
import cv2
import re
from tensorflow.keras.utils import to_categorical

# ✅ Constants for 224x224
IMG_HEIGHT = 224  # Ensure height is 224
IMG_WIDTH = 224   # Ensure width is 224
CHANNELS = 3  # RGB images
NUM_CLASSES = 4  # Brain, CSP, LV, Background

# ✅ Class mapping from RGB to class index
CLASS_MAP = {
    (255, 0, 0): 1,  # Brain
    (0, 255, 0): 2,  # CSP
    (0, 0, 255): 3,  # LV
    (0, 0, 0): 0,  # Background
}

image_dir = r"D:\augmented_dataset\images"
mask_dir = r"D:\augmented_dataset\masks"

# ✅ Define destination directories
train_image_dir = r"D:\Updated\train\images"
train_mask_dir = r"D:\Updated\train\masks"
val_image_dir = r"D:\Updated\val\images"
val_mask_dir = r"D:\Updated\val\masks"
test_image_dir = r"D:\Updated\test\images"
test_mask_dir = r"D:\Updated\test\masks"

# ✅ Fix sorting issue using natural sorting
def natural_sort_key(s):
    """Sort filenames numerically instead of lexicographically."""
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]

# ✅ Convert RGB mask to class index mask
def rgb_to_class(mask_array):
    """Convert RGB mask to single-channel class index mask."""
    height, width, _ = mask_array.shape
    class_mask = np.zeros((height, width), dtype=np.uint8)

    for rgb, class_idx in CLASS_MAP.items():
        matches = np.all(mask_array == rgb, axis=-1)  # Ensure exact match
        class_mask[matches] = class_idx

    return class_mask

# ✅ Preprocess Filtered Dataset for 224x224
def preprocess_filtered_dataset(image_dir, mask_dir):
    """Preprocess images & masks: normalize, resize, and convert masks to one-hot encoding."""

    # ✅ Load and sort filenames correctly
    image_filenames = sorted(os.listdir(image_dir), key=natural_sort_key)
    mask_filenames = sorted(os.listdir(mask_dir), key=natural_sort_key)

    valid_image_paths = []
    valid_mask_paths = []

    # ✅ Ensure each image has a corresponding mask
    for img_file, mask_file in zip(image_filenames, mask_filenames):
        img_path = os.path.join(image_dir, img_file)
        mask_path = os.path.join(mask_dir, mask_file)

        if os.path.exists(img_path) and os.path.exists(mask_path):
            valid_image_paths.append(img_path)
            valid_mask_paths.append(mask_path)
        else:
            print(f"⚠️ Skipping {img_file}: Missing image or mask")

    num_images = len(valid_image_paths)

    # ✅ Initialize arrays
    X = np.zeros((num_images, IMG_HEIGHT, IMG_WIDTH, CHANNELS), dtype=np.float32)
    y = np.zeros((num_images, IMG_HEIGHT, IMG_WIDTH, NUM_CLASSES), dtype=np.float32)  # One-hot encoded masks

    print(f"🚀 Processing {num_images} filtered images and masks...")

    for idx, (img_path, mask_path) in enumerate(zip(valid_image_paths, valid_mask_paths)):
        if idx % 100 == 0:
            print(f"✅ Processed {idx}/{num_images} images")

        # ✅ Load and Resize Image
        img = cv2.imread(img_path)  # Read image in BGR format
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))  # Resize to (224,224)
        img = img.astype(np.float32) / 255.0  # Normalize

        # ✅ Load and Resize Mask
        mask = cv2.imread(mask_path)  # Read mask in BGR format
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        mask = cv2.resize(mask, (IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_NEAREST)  # Resize mask correctly

        # ✅ Convert RGB mask to class mask
        class_mask = rgb_to_class(mask)

        # ✅ One-hot encode the class mask
        one_hot_mask = to_categorical(class_mask, num_classes=NUM_CLASSES)

        # ✅ Store preprocessed data
        X[idx] = img
        y[idx] = one_hot_mask

        # ✅ Clear memory to prevent memory leaks
        del img, mask, class_mask, one_hot_mask
        gc.collect()

    return X, y

from sklearn.model_selection import train_test_split

# ✅ Process dataset splits
X_train, y_train = preprocess_filtered_dataset(train_image_dir, train_mask_dir)
X_val, y_val = preprocess_filtered_dataset(val_image_dir, val_mask_dir)
X_test, y_test = preprocess_filtered_dataset(test_image_dir, test_mask_dir)

# ✅ Print dataset information
print("\n✅ Dataset Splits:")
print(f"  - Training set: {X_train.shape}, {y_train.shape}")
print(f"  - Validation set: {X_val.shape}, {y_val.shape}")
print(f"  - Test set: {X_test.shape}, {y_test.shape}")

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data Augmentation configuration for the training set
train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

# Fit the augmentation parameters on the training data
train_datagen.fit(X_train)

In [ ]:
print("Unique values in y_train:", np.unique(np.argmax(y_train, axis=-1)))
print("Unique values in y_val:", np.unique(np.argmax(y_val, axis=-1)))
print("Unique values in y_test:", np.unique(np.argmax(y_test, axis=-1)))

In [ ]:
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D, UpSampling2D, Concatenate, BatchNormalization, Activation
from tensorflow.keras.applications import ConvNeXtTiny
import gc

# Constants
IMG_HEIGHT = 224
IMG_WIDTH = 224
CHANNELS = 3
NUM_CLASSES = 4

class ResizeLayer(tf.keras.layers.Layer):
    def __init__(self, target_size, **kwargs):
        super(ResizeLayer, self).__init__(**kwargs)
        self.target_size = target_size

    def call(self, inputs):
        return tf.image.resize(inputs, self.target_size, method='bilinear')

    def get_config(self):
        config = super().get_config()
        config.update({"target_size": self.target_size})
        return config

def conv_block(x, filters, kernel_size=3, padding='same', activation='relu'):
    x = Conv2D(filters, kernel_size, padding=padding)(x)
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    x = Conv2D(filters, kernel_size, padding=padding)(x)
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    return x

def build_full_convnexttiny_unet(input_shape=(IMG_HEIGHT, IMG_WIDTH, CHANNELS), num_classes=NUM_CLASSES):
    inputs = Input(shape=input_shape)

    def convnext_preprocess(x):
        mean = tf.constant([0.485, 0.456, 0.406], dtype=tf.float32)
        std = tf.constant([0.229, 0.224, 0.225], dtype=tf.float32)
        return (x / 255.0 - mean) / std

    preprocessed = tf.keras.layers.Lambda(convnext_preprocess)(inputs)

    # Load ConvNeXtTiny
    base_model = ConvNeXtTiny(
        include_top=False,
        weights="imagenet",
        include_preprocessing=False,
        input_shape=input_shape
    )

    # Get skip connection layers by name (from base_model)
    skip_layer_names = [
        "convnext_tiny_stem",
        "convnext_tiny_stage_0_block_2_identity",
        "convnext_tiny_stage_1_block_2_identity",
        "convnext_tiny_stage_2_block_8_identity",
        "convnext_tiny_stage_3_block_2_identity"
    ]
    skip_outputs = [base_model.get_layer(name).output for name in skip_layer_names]
    feature_extractor = Model(inputs=base_model.input, outputs=skip_outputs)

    # Apply the feature extractor to our preprocessed input
    encoder1, encoder2, encoder3, encoder4, encoder5 = feature_extractor(preprocessed)

    bottleneck = Conv2D(512, 1, padding='same')(encoder5)
    bottleneck = BatchNormalization()(bottleneck)
    bottleneck = Activation('relu')(bottleneck)

    up4 = UpSampling2D((2, 2))(bottleneck)
    up4 = ResizeLayer((encoder4.shape[1], encoder4.shape[2]))(up4)
    up4 = conv_block(up4, 512)
    skip4 = Conv2D(256, 1, padding='same')(encoder4)
    skip4 = BatchNormalization()(skip4)
    skip4 = Activation('relu')(skip4)
    merge4 = Concatenate()([up4, skip4])
    merge4 = conv_block(merge4, 384)

    up3 = UpSampling2D((2, 2))(merge4)
    up3 = ResizeLayer((encoder3.shape[1], encoder3.shape[2]))(up3)
    up3 = conv_block(up3, 384)
    skip3 = Conv2D(128, 1, padding='same')(encoder3)
    skip3 = BatchNormalization()(skip3)
    skip3 = Activation('relu')(skip3)
    merge3 = Concatenate()([up3, skip3])
    merge3 = conv_block(merge3, 192)

    up2 = UpSampling2D((2, 2))(merge3)
    up2 = ResizeLayer((encoder2.shape[1], encoder2.shape[2]))(up2)
    up2 = conv_block(up2, 192)
    skip2 = Conv2D(96, 1, padding='same')(encoder2)
    skip2 = BatchNormalization()(skip2)
    skip2 = Activation('relu')(skip2)
    merge2 = Concatenate()([up2, skip2])
    merge2 = conv_block(merge2, 96)

    up1 = UpSampling2D((2, 2))(merge2)
    up1 = ResizeLayer((encoder1.shape[1], encoder1.shape[2]))(up1)
    up1 = conv_block(up1, 96)
    skip1 = Conv2D(48, 1, padding='same')(encoder1)
    skip1 = BatchNormalization()(skip1)
    skip1 = Activation('relu')(skip1)
    merge1 = Concatenate()([up1, skip1])
    merge1 = conv_block(merge1, 48)

    up_final = UpSampling2D((2, 2))(merge1)
    up_final = conv_block(up_final, 32)
    if up_final.shape[1] != input_shape[0] or up_final.shape[2] != input_shape[1]:
        up_final = ResizeLayer((input_shape[0], input_shape[1]))(up_final)

    outputs = Conv2D(num_classes, 1, activation='softmax', dtype='float32')(up_final)
    model = Model(inputs=inputs, outputs=outputs)
    return model, base_model

# Create the model
print("Creating full ConvNeXtTiny-UNet model...")
model, base_model = build_full_convnexttiny_unet()
print("\nFull UNet Model Summary:")
model.summary()

# Clear memory
gc.collect()
tf.keras.backend.clear_session()

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ✅ Dice Coefficient Metric
def dice_coefficient(y_true, y_pred):
    smooth = 1e-15
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    intersection = tf.reduce_sum(y_true * y_pred, axis=[1,2,3])
    union = tf.reduce_sum(y_true, axis=[1,2,3]) + tf.reduce_sum(y_pred, axis=[1,2,3])
    dice = (2. * intersection + smooth) / (union + smooth)
    return tf.reduce_mean(dice)

# ✅ Weighted Categorical Crossentropy
def weighted_categorical_crossentropy(y_true, y_pred):
    class_weights = tf.constant([0.3776, 0.7605, 65.8554, 46.2381], dtype=tf.float32)
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.clip_by_value(y_pred, K.epsilon(), 1.0)
    loss = -tf.reduce_sum(y_true * tf.math.log(y_pred), axis=-1)
    class_weights = tf.reshape(class_weights, (1, 1, 1, NUM_CLASSES))
    weighted_loss = loss * tf.reduce_sum(class_weights, axis=-1)
    return tf.reduce_mean(weighted_loss)

# ✅ Dice Loss
def dice_loss(y_true, y_pred):
    smooth = 1e-6
    y_true = tf.cast(y_true, y_pred.dtype)
    intersection = tf.reduce_sum(y_true * y_pred, axis=[1, 2])
    union = tf.reduce_sum(y_true, axis=[1, 2]) + tf.reduce_sum(y_pred, axis=[1, 2])
    dice = (2. * intersection + smooth) / (union + smooth)
    return 1 - tf.reduce_mean(dice)

# ✅ Combined Loss Function
def combined_loss(y_true, y_pred):
    return weighted_categorical_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

# ✅ Custom Dice Coefficient Metric for Each Class
class DiceCoefficient(tf.keras.metrics.Metric):
    def __init__(self, class_idx, name=None, **kwargs):  
        if name is None:
            name = f"DiceClass{class_idx}"  
        super(DiceCoefficient, self).__init__(name=name, **kwargs)
        self.class_idx = class_idx
        self.dice = self.add_weight(name="dice", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true_class = y_true[..., self.class_idx]
        y_pred_class = y_pred[..., self.class_idx]
        intersection = tf.reduce_sum(y_true_class * y_pred_class, axis=[1, 2])
        union = tf.reduce_sum(y_true_class, axis=[1, 2]) + tf.reduce_sum(y_pred_class, axis=[1, 2])
        dice = (2. * intersection + 1e-6) / (union + 1e-6)
        self.dice.assign(tf.reduce_mean(dice))

    def result(self):
        return self.dice

# ✅ Function to Get Class-wise Metrics
def class_wise_metrics(num_classes=4):
    return [DiceCoefficient(i) for i in range(num_classes)] + [tf.keras.metrics.MeanIoU(num_classes=num_classes)]

# ✅ Create Data Generator
def create_train_generator(X, y, batch_size=16):
    data_gen_args = dict(
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    
    image_datagen = ImageDataGenerator(**data_gen_args)
    mask_datagen = ImageDataGenerator(**data_gen_args)
    
    seed = 42
    image_generator = image_datagen.flow(X, batch_size=batch_size, seed=seed)
    mask_generator = mask_datagen.flow(y, batch_size=batch_size, seed=seed)
    
    while True:
        X_batch = next(image_generator)
        y_batch = next(mask_generator)
        yield X_batch, y_batch

# ✅ Create the generator
train_generator = create_train_generator(X_train, y_train, batch_size=8)

# ✅ Compile the Model
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss=combined_loss,
    metrics=class_wise_metrics(4)
)

# ✅ Train the Model with the Data Generator
history = model.fit(
    train_generator,
    steps_per_epoch=len(X_train) // 8,
    validation_data=(X_val, y_val),
    epochs=100,
    callbacks=[
        EarlyStopping(
            monitor='val_loss', 
            patience=7, 
            restore_best_weights=True
        ),
        ReduceLROnPlateau(
            monitor='val_loss', 
            factor=0.5, 
            patience=3, 
            min_lr=1e-6
        ),
        ModelCheckpoint(
            'best_Unet++_ConvNeXtTiny.keras',
            monitor='val_loss',
            save_best_only=True
        )
    ]
)

In [ ]:
import matplotlib.pyplot as plt

# Extracting data from the history object
history_dict = history.history

# Plotting the training and validation loss
plt.figure(figsize=(12, 6))

# Plotting loss
plt.subplot(1, 2, 1)
plt.plot(history_dict['loss'], label='Training Loss')
plt.plot(history_dict['val_loss'], label='Validation Loss')
plt.title('Loss Curves')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# If accuracy is available, plot training and validation accuracy
if 'accuracy' in history_dict:
    plt.subplot(1, 2, 2)
    plt.plot(history_dict['accuracy'], label='Training Accuracy')
    plt.plot(history_dict['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy Curves')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from scipy.spatial.distance import directed_hausdorff
from tensorflow.keras.utils import to_categorical

# ✅ RGB to Class Index Conversion (for the test masks)
RGB_TO_CLASS = {
    (255, 0, 0): 1,  # Brain
    (0, 255, 0): 2,  # CSP
    (0, 0, 255): 3,  # LV
    (0, 0, 0): 0     # Background
}

# ✅ Function to convert RGB masks to class index masks
def rgb_to_class_mask(rgb_mask):
    # Create a mask initialized with zeros (for background class)
    class_mask = np.zeros(rgb_mask.shape[:2], dtype=int)

    # Loop through the RGB_TO_CLASS dictionary
    for rgb, class_idx in RGB_TO_CLASS.items():
        # Identify the pixels with the current RGB value and assign them the class index
        match_mask = np.all(rgb_mask == np.array(rgb), axis=-1)
        class_mask[match_mask] = class_idx

    return class_mask

# ✅ Function to calculate Dice Similarity Coefficient (DSC)
def dice_coefficient(y_true, y_pred):
    smooth = 1e-15
    intersection = np.sum(y_true * y_pred)
    union = np.sum(y_true) + np.sum(y_pred)
    return (2. * intersection + smooth) / (union + smooth)

# ✅ Function to calculate IoU (Intersection over Union)
def iou(y_true, y_pred):
    intersection = np.sum(y_true * y_pred)
    union = np.sum(y_true) + np.sum(y_pred) - intersection
    return intersection / (union + 1e-15)

# ✅ Function to calculate Hausdorff Distance
def hausdorff_distance(y_true, y_pred):
    true_points = np.array(np.where(y_true == 1)).T
    pred_points = np.array(np.where(y_pred == 1)).T

    if len(true_points) == 0 or len(pred_points) == 0:
        return float('inf')  # Return inf if no points for either true or pred class

    forward_hausdorff = directed_hausdorff(true_points, pred_points)[0]
    reverse_hausdorff = directed_hausdorff(pred_points, true_points)[0]
    return max(forward_hausdorff, reverse_hausdorff)

# ✅ Function to calculate Average Surface Distance (ASD)
def average_surface_distance(y_true, y_pred):
    true_points = np.array(np.where(y_true == 1)).T
    pred_points = np.array(np.where(y_pred == 1)).T

    if len(true_points) == 0 or len(pred_points) == 0:
        return float('inf')  # Return inf if no points for either true or pred class

    distances = []
    for true_point in true_points:
        distances.append(np.min(np.linalg.norm(pred_points - true_point, axis=1)))
    return np.mean(distances)

# ✅ Function to evaluate the model on the test set class-wise
def evaluate_classwise_metrics(model, X_test, y_test, num_classes=4, batch_size=16):
    # Predict in batches
    y_pred = model.predict(X_test, batch_size=batch_size)
    y_pred = np.argmax(y_pred, axis=-1)  # Convert to class index prediction

    # Convert y_test to class index format (since it's one-hot encoded)
    y_test_class = np.argmax(y_test, axis=-1)

    # Initialize lists to store class-wise metrics
    class_metrics = {i: {'dice': [], 'iou': [], 'precision': [], 'recall': [], 'f1': [], 'accuracy': [], 'hausdorff': [], 'asd': []} for i in range(num_classes)}

    # Calculate metrics for each test sample
    for i in range(len(X_test)):
        true_mask = y_test_class[i]  # one-hot -> class index
        pred_mask = y_pred[i]

        # For each class (0: Background, 1: Brain, 2: CSP, 3: LV)
        for class_idx in range(num_classes):
            true_class_mask = (true_mask == class_idx).astype(int)
            pred_class_mask = (pred_mask == class_idx).astype(int)

            # Dice Coefficient
            class_metrics[class_idx]['dice'].append(dice_coefficient(true_class_mask, pred_class_mask))
            # IoU
            class_metrics[class_idx]['iou'].append(iou(true_class_mask, pred_class_mask))
            # Precision
            class_metrics[class_idx]['precision'].append(precision_score(true_class_mask.flatten(), pred_class_mask.flatten(), zero_division=0))
            # Recall
            class_metrics[class_idx]['recall'].append(recall_score(true_class_mask.flatten(), pred_class_mask.flatten(), zero_division=0))
            # F1 Score
            class_metrics[class_idx]['f1'].append(f1_score(true_class_mask.flatten(), pred_class_mask.flatten(), zero_division=0))
            # Accuracy
            class_metrics[class_idx]['accuracy'].append(accuracy_score(true_class_mask.flatten(), pred_class_mask.flatten()))
            # # Hausdorff Distance
            # class_metrics[class_idx]['hausdorff'].append(hausdorff_distance(true_class_mask, pred_class_mask))
            # # Average Surface Distance
            # class_metrics[class_idx]['asd'].append(average_surface_distance(true_class_mask, pred_class_mask))

    # Print class-wise metrics in percentage
    print(f"{'Class':<10}{'Dice Coefficient (%)':<20}{'IoU (%)':<20}{'Precision (%)':<20}{'Recall (%)':<20}{'F1 Score (%)':<20}{'Accuracy (%)':<20}{'Hausdorff Distance':<20}{'Avg Surface Distance':<20}")
    print('-' * 180)

    for class_idx in range(num_classes):
        print(f"Class {class_idx}:")
        print(f"  Dice Coefficient: {np.mean(class_metrics[class_idx]['dice']) * 100:.2f}%")
        print(f"  IoU: {np.mean(class_metrics[class_idx]['iou']) * 100:.2f}%")
        print(f"  Precision: {np.mean(class_metrics[class_idx]['precision']) * 100:.2f}%")
        print(f"  Recall: {np.mean(class_metrics[class_idx]['recall']) * 100:.2f}%")
        print(f"  F1 Score: {np.mean(class_metrics[class_idx]['f1']) * 100:.2f}%")
        print(f"  Accuracy: {np.mean(class_metrics[class_idx]['accuracy']) * 100:.2f}%")
        # print(f"  Hausdorff Distance: {np.mean(class_metrics[class_idx]['hausdorff']):.4f}")
        # print(f"  Average Surface Distance: {np.mean(class_metrics[class_idx]['asd']):.4f}")
        print("-" * 180)

    # Evaluate on test set to print overall test accuracy and loss
    test_loss, *test_metrics = model.evaluate(X_test, y_test, batch_size=batch_size)
    print(f"Test Loss: {test_loss:.4f}")

    for metric, value in zip(model.metrics_names[1:], test_metrics):
        print(f"{metric}: {value:.4f}")

# ✅ Call the evaluation function on the test set class-wise
evaluate_classwise_metrics(model, X_test, y_test)